In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from data import FocusDataset

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

2025-05-21 16:02:12.681100: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 16:02:12.827164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747810932.879920  207625 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747810932.896346  207625 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747810933.013100  207625 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
train_data_list = ['00_01', '00_02', '00_03', '00_04', '00_05', '10_02', '10_03']
valid_data_list = ['00_01', '00_02']

In [3]:
base_path = '/shared_data/focussu/109.학습태도_및_성향_관찰_데이터/3.개방데이터/1.데이터'
train_dataset = FocusDataset(base_path+'/Training', train_data_list)
test_dataset = FocusDataset(base_path+'/Validation', valid_data_list)
print(len(train_dataset))


pkl_files: 229384
meta_files: 229384
메타데이터 로딩 중...


메타데이터: 100%|██████████| 229384/229384 [00:49<00:00, 4670.42it/s]


pkl_files: 1717
meta_files: 1717
메타데이터 로딩 중...


메타데이터: 100%|██████████| 1717/1717 [00:00<00:00, 4715.26it/s]

229384


## DL Model

In [4]:
class PointNetClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(PointNetClassifier, self).__init__()

        self.fc1 = nn.Linear(3, 64)
        self.bn1 = nn.BatchNorm1d(64)

        self.fc2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)

        self.fc3 = nn.Linear(128, 256)
        self.bn3 = nn.BatchNorm1d(256)

        self.cls_fc1 = nn.Linear(256, 128)
        self.cls_bn1 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.3)
        self.cls_fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        B, N, D = x.size()  # (B, 478, 3)

        x = self.fc1(x)  # (B, N, 64)
        x = self.bn1(x.transpose(1, 2)).transpose(1, 2)
        x = F.relu(x)

        x = self.fc2(x)  # (B, N, 128)
        x = self.bn2(x.transpose(1, 2)).transpose(1, 2)
        x = F.relu(x)

        x = self.fc3(x)  # (B, N, 256)
        x = self.bn3(x.transpose(1, 2)).transpose(1, 2)
        x = F.relu(x)

        x = torch.max(x, dim=1)[0]  # (B, 256)

        x = self.cls_fc1(x)         # (B, 128)
        x = self.cls_bn1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.cls_fc2(x)         # (B, num_classes)
        return x


In [5]:
train_loader = DataLoader(train_dataset, num_workers=3, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, num_workers=2, batch_size=32, shuffle=False)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PointNetClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, batch in pbar:
        x = batch['landmarks'].to(device)
        y = (batch['label']['category_id'] - 1).long().to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()       
        
    if(epoch % 10 == 0):
        print(f'Epoch {epoch} Loss {loss.item()}')
    #     model.eval()
    #     test_loss = 0
    #     correct = 0
    #     total = 0
    #     with torch.no_grad():
    #         for batch_idx, (data, target) in enumerate(test_loader):
    #             data, target = data.to(device), target.to(device)
    #             output = model(data)
    #             loss = criterion(output, target)
    #             test_loss += loss.item()
    #             _, predicted = output.max(1)
    #             total += target.size(0)

100%|██████████| 7169/7169 [01:00<00:00, 119.22it/s]

Epoch 0 Loss 0.2081659585237503



100%|██████████| 7169/7169 [00:59<00:00, 120.50it/s]


Epoch 10 Loss 0.0009087839280255139


100%|██████████| 7169/7169 [00:57<00:00, 123.65it/s]

...


100%|██████████| 7169/7169 [00:54<00:00, 130.65it/s]

Epoch 80 Loss 0.06166066601872444



100%|██████████| 7169/7169 [00:55<00:00, 128.59it/s]

Epoch 90 Loss 0.01797585003077984



100%|██████████| 7169/7169 [00:58<00:00, 122.12it/s]


In [7]:
# 모델 저장
model_save_path = '/home/hyun/focussu-ai/model/landmark_classifier_v1.pkl'
torch.save(model.state_dict(), model_save_path)
print(f'모델이 {model_save_path}에 저장되었습니다.')


모델이 /home/hyun/focussu-ai/model/landmark_classifier_v1.pkl에 저장되었습니다.
